In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip install JPype1
!pip install rhinoMorph

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [14]:
#Program FIiles 처럼 띄어 쓰기 가 되어있으면 인식을 못한다. Program\ FIiles
%cd /content/gdrive/My Drive

/content/gdrive/My Drive


In [15]:
def read_data(filename, encoding='cp949'): # 읽기 함수 정의
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:] # txt 파일의 헤더(id document label)는 제외하기
  return data

def write_data(data, filename, encoding='cp949'): # 쓰기 함수도 정의
  with open(filename, 'w', encoding=encoding) as f:
    f.write(data)

data = read_data('/content/gdrive/My Drive/2021-07-27-20-41-49-블랙위도우 평점리뷰.txt' , encoding='utf-8') 

In [16]:
import rhinoMorph
rn = rhinoMorph.startRhino()

morphed_data = ''
for data_each in data:
  morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each[1], pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
  joined_data_each = ' '.join(morphed_data_each) # 문자열을 하나로 연결
  if joined_data_each: # 내용이 있는 경우만 저장하게 함
    morphed_data += data_each[0]+"\t"+joined_data_each+"\t"+data_each[2]+"\n"

# 형태소 분석된 파일 저장
write_data(morphed_data, 'review_morphed_210727.txt', encoding='cp949')

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!


In [17]:
# 형태분석된 데이터 로딩
data = read_data('review_morphed_210727.txt' , encoding='cp949')
print(len(data)) # 개수
print(len(data[0])) 
print(data[0]) #[인덱스번호, 내용, 평점]

4307
3
['1', '역시 마블 영화관 이렇게 가슴 웅장 해진 오랜만', '1']


In [18]:
%cd /content/gdrive/My\ Drive/pytest

/content/gdrive/My Drive/pytest


In [19]:
# 감정사전 읽기
data_id = [line[0] for line in data] # 데이터 id
data_text = [line[1] for line in data] # 데이터 본문
data_senti = [line[2] for line in data] # 데이터 긍부정 부분

positive = read_data('positive.txt') # 긍정 감정사전 읽기
negative = read_data('negative.txt') # 부정 감정사전 읽기

In [20]:
# 훈련데이터와 테스트데이터 분리 (자동)
data_text = [line[1] for line in data] # 데이터 본문
data_senti = [line[2] for line in data] # 데이터 긍부정 부분

from sklearn.model_selection import train_test_split # 본문과 라벨을 각각 분리
train_data_text, test_data_text, train_data_senti, test_data_senti = train_test_split(data_text, data_senti, stratify=data_senti)

# Counter 클래스를 이용해 각 분류가 훈련데이터와 테스트데이터에 같은 비율로 들어갔는지 확인해 본다
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq:', train_data_senti_freq)
test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq:', test_data_senti_freq)


train_data_senti_freq: Counter({'1': 2764, '0': 466})
test_data_senti_freq: Counter({'1': 921, '0': 156})


In [21]:
# 행렬 형태로 변환
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5).fit(train_data_text) # 최소 문서 빈도 5이상의 단어만 대상
X_train = vect.transform(train_data_text) # 행렬 생성
print("X_train:\n", repr(X_train)) # 생성된 행렬 개요

X_train:
 <3230x682 sparse matrix of type '<class 'numpy.int64'>'
	with 19915 stored elements in Compressed Sparse Row format>


In [22]:
# 행렬 내용 관찰
feature_names = vect.get_feature_names()
print("특성 개수:", len(feature_names))
print("처음 20개 특성:\m", feature_names[:20])
print("3000~5000까지의 특성:\n", feature_names[3000:5000])

특성 개수: 682
처음 20개 특성:\m ['10점', 'cgv', 'pc', 'ㄷㄷ', 'ㅇㅇ', 'ㅋㅋ', 'ㅎㅎ', 'ㅜㅜ', 'ㅠㅠ', 'ㅡㅡ', '가능', '가득', '가디언', '가볍', '가슴', '가장', '가족', '가족애', '가족영화', '가지']
3000~5000까지의 특성:
 []


In [23]:
# 머신러닝 알고리즘 적용
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
y_train = pd.Series(train_data_senti) # 리스트 형태를 종속변수가 될 수 있는 1차원 배열(Series)로 만든다

lr = LogisticRegression(solver="liblinear") # 모델 생성
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
# 테스트 데이터 입력
X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)
print("테스트 데이터 점수:", lr.score(X_test, y_test))

테스트 데이터 점수: 0.9173630454967502


In [27]:
# 1개 데이터 예측
# 형태소분석기 시작
import rhinoMorph
rn = rhinoMorph.startRhino()
new_input = '지민이는 졸업 작품을 성공적으로 끝냈다'
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input)
inputdata.append(morphed_input)
print('input data:', inputdata)

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
input data: ['지민 졸업 작품 성공적 끝내']


In [28]:
X_input = vect.transform(inputdata) # 앞에서 만든 11445 컬럼의 행렬에 적용
result = lr.predict(X_input) # 0은 부정,1은 긍정
if result == "0" : # 문자열 형태로 출력된다
  print("부정적인 글입니다")
else:
  print("긍정적인 글입니다")

긍정적인 글입니다
